In [ ]:
#%pip install fastapi uvicorn pydantic langchain langchain_experimental langchain_huggingface langchain_ollama chromadb langchain_community

In [ ]:
from fastapi import FastAPI
import chromadb
from sentence_transformers import SentenceTransformer
from pydantic import BaseModel

# 📌 Init FastApi
app = FastAPI()

# 📌 Embedding Model that we have created before
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 📌 Init ChromaDB / Read the vector_db
chroma_client = chromadb.PersistentClient(path="chroma_db")
faq_collection = chroma_client.get_or_create_collection(name="faq_ecommerce")

# 📌 API will get a JSON with "question"
class QueryRequest(BaseModel):
    question: str

# 📌 Función de búsqueda en ChromaDB
def search_faq(query, top_k=1):
    query_embedding = embedding_model.encode(query, convert_to_numpy=True).tolist()
    results = faq_collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    if results["ids"]:
        best_match = results["metadatas"][0][0]  # Extrae la mejor coincidencia
        return {"answer": best_match["answer"]}
    else:
        return {"answer": "Sorry, I couldn't find an answer to your question."}

# 📌 Endpoint para recibir preguntas
@app.post("/ask")
def ask_question(request: QueryRequest):
    return search_faq(request.question)

# 📌 Endpoint de prueba
@app.get("/")
def home():
    return {"message": "E-commerce FAQ Chatbot API is running!"}
